In [ ]:
#Chap 16.휴넷 사이트에서 특정 강의 리뷰 정보 수집하기

#Lesson 1 - 필요한 모듈 불러오고 필요정보 입력 받은 후 자동 검색 수행
#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time    
import math

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 연습문제: 이 크롤러는 휴넷 사이트의 강의 리뷰 수집용 웹크롤러입니다.")
print("=" *100)

try :
    cnt = int( input('1.수집할 리뷰는 총 몇건입니까?(기본값:10): '))
except ValueError :
    cnt = 10
    print('기본값인 10 건으로 수집을 진행합니다.')
page_cnt = math.ceil( cnt / 10)
print()

#Step 3. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://hbs.hunet.co.kr/Education/Detail?gid=Y00079478'
driver.get(url)
driver.maximize_window()
time.sleep(3)

#오른쪽 상단의 Best 수강후기의 더보기 버튼 클릭
driver.find_element(By.XPATH,'//*[@id="content-wide"]/div[1]/div[2]/div[3]/div[1]/div[3]/a').click()
time.sleep(1)


#Lesson 2. 각 항목별 상세 데이터를 수집하기
# Step 6.각 데이터 저장용 리스트 생성
no2 = [ ]           # 게시글 번호 컬럼
writer2 = [ ]       # 리뷰 작성자 제목 컬럼
writer_id2 = [ ]    # 리뷰 작성자 ID 컬럼
review2 = [ ]       # 리뷰 내용 컬럼

# Step 7. 주요 내용을 추출하여 리스트에 저장
from bs4 import BeautifulSoup

no = 1  # 게시물 번호용 변수

for pno in range(1,page_cnt+1) :
    print('%s 페이지의 리뷰 정보를 수집합니다======' %pno)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_list = soup.find('ul','view-epilogue-list').find_all('li')

    for i in review_list :
        print('%s번째 리뷰 정보를 수집합니다=======' %no)
        # 게시물 번호 리스트에 추가
        no2.append(no)                            
        print('1.번호:',no)

        review_all = i.find('div','author').find_all('strong')
        

In [3]:
type(review_all)

bs4.element.ResultSet

In [1]:
        #리뷰 작성자 이름 정보
        writer = review_all[0].get_text().strip()
        writer2.append(writer)                      
        print('2.리뷰 작성자 이름:',writer)

        #리뷰 작성자 이름 정보
        writer_id = review_all[1].get_text().strip()
        writer_id2.append(writer_id)                      
        print('3.리뷰 작성자 ID:',writer_id)

        # 리뷰 내용
        try :
            i.strong.decompose()   #strong 태그 제거
        except :
            pass
        else :
            i.strong.decompose()   #strong 태그 제거
            review = i.get_text().replace('\n','').strip()
            review2.append(review)
            print('4.리뷰 내용:', review)

        print("\n")

        no += 1
        
        if no > cnt :
            break
            
    time.sleep(1)    
    pno += 1
    try :
        driver.find_element(By.LINK_TEXT,'%s' %pno).click()
    except :
        driver.find_element(By.LINK_TEXT,'»').click()
    

#Lesson 3.수집한 데이터를 다양한 형식의 파일로 저장하기
# Step 8.출력 결과를 표(데이터 프레임) 형태로 만들기
import openpyxl      # pip install openpyxl
import pandas as pd  # pip install pandas
import os

hunet = pd.DataFrame()
hunet['번호'] = no2
hunet['작성자이름'] = pd.Series(writer2)
hunet['작성자ID'] =pd.Series(writer_id2)
hunet['리뷰내용'] =pd.Series(review2)

# Step 9. 저장될 파일위치와 이름을 지정한 후 csv , xls 파일로 저장하기
f_dir = input('파일을 저장할 폴더 이름을 쓰세요(기본경로:c:\\py_temp\\):')
if f_dir=='' :
    f_dir='c:\\py_temp\\'
    
query_txt='리뷰수집'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

sec_name = 'hunet_review'
os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xlsx'

# csv 형태로 저장하기
hunet.to_csv(fc_name,encoding="utf-8-sig",index=False)
print("1.csv 파일 저장 경로: %s" %fc_name) 

# 엑셀 형태로 저장하기
hunet.to_excel(fx_name , index=False , engine='openpyxl')
print("2.xlsx 파일 저장 경로: %s" %fx_name) 

 연습문제: 이 크롤러는 휴넷 사이트의 강의 리뷰 수집용 웹크롤러입니다.
1.수집할 리뷰는 총 몇건입니까?(기본값:10): 10

1 페이지의 리뷰 정보를 수집합니다======
1번째 리뷰 정보를 수집합니다=======
1.번호: 1
2.리뷰 작성자 이름: 한*우
3.리뷰 작성자 ID: lgi****
4.리뷰 내용: 요즘 많은 분들이 개발자에 대한 관심이 많은 걸로 알고 있습니다. 저 또한 그런 사람 중에 하나였습니다. 휴넷에 파이썬에 대한 강의가 있는 걸 보고 바로 신청했습니다. 사실 파이썬에 대해 전혀 몰라서 강의를 신청하고도 걱정이 좀 됐습니다. 그렇게 시작한 강의는 제 걱정을 싹 해결해줬습니다. 파이썬이란 프로그램이 어떤 위치에서 어떻게 작용하는지의 아주 기본적인 부분부터 강사님께서 세세하게 설명해주셔서 전체적인 개념을 잡을 수 있었고, 본격적인 코딩에 대한 설명도 이해하기 쉽게 설명해주시고 강의 자체가 짧게 되어있다 보니 학습하고 부담없이 복습도 할 수 있었습니다. 너무 강추하는 강의입니다. 꼭 들어보세요.


2번째 리뷰 정보를 수집합니다=======
1.번호: 2
2.리뷰 작성자 이름: 
3.리뷰 작성자 ID: met****
4.리뷰 내용: 코딩이 이슈가 되면서 파이썬을 한번쯤 배워보고 싶다고 막연하게만 생각하고 있었습니다. 그러던 도중 이번 강의에 파이썬 강의가 열린 것을 보고 신청했습니다! 기초적인 기본 개념부터 시작해서 쉬운 프로그래밍 방법, 그리고 약간의 심화내용까지 실습과정을 계속 보여주어서 문과생인 저도 잘 이해하고 따라갈 수 있었고, 더욱 심화 과정을 공부해보고 싶다는 생각도 갖게 되었습니다. 만족스러웠던 파이썬 프로그래밍 강의였습니다~!


3번째 리뷰 정보를 수집합니다=======
1.번호: 3
2.리뷰 작성자 이름: 김*무
3.리뷰 작성자 ID: dwf****
4.리뷰 내용: 한번 배우면 두고두고 사용할 수 있을만한 것이 무엇인지 고민해보다가 코딩이란 것을 처음 배워보았습니다. 교육 내용 그리 길지 않습니다. 짧고 굵게 이 영상 